In [1]:
# Imports
import pandas as pd  # type: ignore
import numpy as np  # type: ignore
import base64
from utils import (
    get_rules_list,
    create_train_test_split,
    create_model,
    payload_to_vec,
    create_adv_train_test_split,
    test_evasion,
)
from modsec import init_modsec

from sklearn.ensemble import RandomForestClassifier  # type: ignore
from wafamole.evasion import EvasionEngine  # type: ignore

In [2]:
# Set up variables

attack_data_path = "data/attacks_full.sql"
sane_data_path = "data/sanes_full.sql"

rule_ids = get_rules_list()
modsec = init_modsec()

In [3]:
# Create train and test sets and train model

paranoia_level = 4

train, test = create_train_test_split(
    attack_file=attack_data_path,
    sane_file=sane_data_path,
    train_attacks_size=5000,  # paper uses 10000
    train_sanes_size=5000,  # paper uses 10000
    test_attacks_size=1000,  # paper uses 2000
    test_sanes_size=1000,  # paper uses 2000
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)
train.to_csv("data/train_5k.csv", index=False)
test.to_csv("data/test_1k.csv", index=False)

# # load the train and test sets from disk
# train = pd.read_csv("data/train_5k.csv")
# test = pd.read_csv("data/test_1k.csv")
# train['vector'] = train['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
# test['vector'] = test['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

wafamole_model, threshold = create_model(
    train=train,
    test=test,
    model=RandomForestClassifier(n_estimators=160, random_state=666),
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.97      0.94      0.95       500
        sane       0.94      0.97      0.95       500

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000

Adjusted threshold: 0.8027458911979671
              precision    recall  f1-score   support

      attack       0.85      0.99      0.92       500
        sane       0.99      0.83      0.90       500

    accuracy                           0.91      1000
   macro avg       0.92      0.91      0.91      1000
weighted avg       0.92      0.91      0.91      1000



In [4]:
# adversarial training

engine = EvasionEngine(wafamole_model)
train_adv, test_adv = create_adv_train_test_split(
    train=train,
    test=test,
    train_adv_size=250, # paper uses 5000 (1/4 of total train set size)
    test_adv_size=100, # paper uses 2000 (1/2 of total test set size)
    engine=engine,
    engine_settings={
        "max_rounds": 200,
        "round_size": 10,
        "timeout": 10,
        "threshold": threshold,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)
train_adv.to_csv("data/train_adv_50.csv", index=False)
test_adv.to_csv("data/test_adv_50.csv", index=False)

# # load the train_adv and test_adv sets from disk
# train_adv = pd.read_csv("data/train_adv_50.csv")
# test_adv = pd.read_csv("data/test_adv_50.csv")
# train_adv['vector'] = train_adv['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
# test_adv['vector'] = test_adv['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

# train new model with train + train_adv
wafamole_model_adv, threshold_adv = create_model(
    train=pd.concat([train, train_adv]).sample(frac=1).reset_index(drop=True),
    test=pd.concat([test, test_adv]).sample(frac=1).reset_index(drop=True),
    model=RandomForestClassifier(n_estimators=160, random_state=666),
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.97      0.94      0.96       550
        sane       0.94      0.97      0.95       500

    accuracy                           0.95      1050
   macro avg       0.95      0.95      0.95      1050
weighted avg       0.95      0.95      0.95      1050

Adjusted threshold: 0.8192005703746961
              precision    recall  f1-score   support

      attack       0.86      0.99      0.92       550
        sane       0.99      0.83      0.90       500

    accuracy                           0.91      1050
   macro avg       0.93      0.91      0.91      1050
weighted avg       0.92      0.91      0.91      1050



In [5]:
# Test the model (without adversarial training)
test_evasion(
    payload='SELECT SLEEP(5)#";',
    threshold=threshold,
    model=wafamole_model,
    engine=EvasionEngine(wafamole_model),
    engine_eval_settings={
        "max_rounds": 200,
        "round_size": 10,
        "timeout": 60,
        "threshold": 0.0,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.96247
[!] Max number of iterations reached
Reached confidence 0.6410169402648634
with payload
sElEcTsleEP(0x5) OR/*!*/0x2542 NOT  LIKE  0x2542 || False#";vE

Min payload: b'sElEcT\x0csleEP(0x5) OR/*!*/0x2542 NOT  LIKE  0x2542 || False#";vE'
Min confidence: 0.64102
Reduced confidence from 0.96247 to 0.64102 (reduction of 0.32145)

Evasion successful


In [7]:
# Test the model (with adversarial training)
test_evasion(
    payload='SELECT SLEEP(5)#";',
    threshold=threshold,
    model=wafamole_model_adv,
    engine=EvasionEngine(wafamole_model_adv),
    engine_eval_settings={
        "max_rounds": 200,
        "round_size": 10,
        "timeout": 60,
        "threshold": 0.0,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.98125
[!] Max number of iterations reached
Reached confidence 0.7767581862544739
with payload
SeleCt SlEEp(0X5)
Or/**/1605 NoT  LiKe  0x645 || (SELECT 8659)   NOT LIKE   8659#";

Min payload: b'SeleCt SlEEp(0X5)\nOr/**/1605 NoT  LiKe  0x645 || (SELECT 8659)   NOT LIKE   8659#";'
Min confidence: 0.77676
Reduced confidence from 0.98125 to 0.77676 (reduction of 0.20449)

Evasion successful


: 